In [9]:
pip install langchain langchain_groq

In [1]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from getpass import getpass

# Hidden input field
groq_key = getpass("Enter your GROQ API key: ")

# Store in environment variable for LangChain
os.environ["GROQ_API_KEY"] = groq_key

llm = ChatGroq(model="llama-3.1-8b-instant")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{question}")
])

chain = prompt | llm

result = chain.invoke({
    "question": "Write a short, suspenseful thriller story about a lost boy trapped in a haunted mansion as he searches for his missing parents."
})

print(result.content)

Enter your GROQ API key: ··········
**The Shadows of Blackwood Manor**

As the sun dipped below the horizon, 12-year-old Jack Harris found himself alone and lost in the labyrinthine corridors of Blackwood Manor. He had been exploring the mansion with his parents, Dr. Emma and Professor Ryan, just hours before. But in the chaos of their investigation, Jack had wandered off in search of a rare book in the library. Now, he was trapped, and his parents were nowhere to be found.

Panic set in as Jack's footsteps echoed through the empty halls. He frantically rushed from room to room, calling out for his parents, but his voice was swallowed by the thick, dusty air. The manor seemed to be shifting and twisting around him, making it impossible to keep track of which direction he had come from.

As the darkness deepened, Jack began to feel a presence lurking just out of sight. He tried to shake off the feeling, telling himself it was just his imagination, but the hairs on the back of his neck r

LangGraph Implementation


In [11]:
# =========================
# LANGGRAPH IMPLEMENTATION
# =========================

from typing import TypedDict
from langgraph.graph import StateGraph, END

# Define Graph State
class AgentState(TypedDict):
    question: str
    refined_question: str
    answer: str

# Node 1: Refine Question
def refine_node(state: AgentState):
    refinement_prompt = f"Improve and clarify this user query:\n{state['question']}"
    result = llm.invoke(refinement_prompt)
    return {"refined_question": result.content}

# Node 2: Generate Final Answer
def answer_node(state: AgentState):
    final_prompt = f"""
    You are an advanced AI assistant.
    Answer the following clearly and professionally:

    Question: {state['refined_question']}
    """
    result = llm.invoke(final_prompt)
    return {"answer": result.content}

# Build Graph
builder = StateGraph(AgentState)

builder.add_node("refiner", refine_node)
builder.add_node("answer_generator", answer_node)

builder.set_entry_point("refiner")
builder.add_edge("refiner", "answer_generator")
builder.add_edge("answer_generator", END)

graph = builder.compile()

# Execute Graph
response = graph.invoke({
    "question": "Explain AI in simple terms"
})

print(response["answer"])

I'd be happy to explain artificial intelligence (AI) in simple terms.

**What is Artificial Intelligence (AI)?**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. Learning
2. Problem-solving
3. Reasoning
4. Perception
5. Understanding natural language

In other words, AI is a branch of computer science that aims to create machines that can think and act like humans.

**How does AI work?**

AI works by using complex algorithms and data analysis to enable machines to learn from experience, make decisions, and improve their performance over time. Here's a high-level overview of the AI process:

1. **Data collection**: AI systems collect data from various sources, such as sensors, databases, or user input.
2. **Data analysis**: AI algorithms process the collected data to identify patterns, relationships, and insights.
3. **Model training**: AI systems use the analyzed data to train mach

LangGraph With Memory

In [12]:
# =========================
# LANGGRAPH WITH MEMORY
# =========================

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph_with_memory = builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "session_1"}}

response1 = graph_with_memory.invoke(
    {"question": "What is Machine Learning?"},
    config=config
)

response2 = graph_with_memory.invoke(
    {"question": "Explain it in 2 lines"},
    config=config
)

print(response2["answer"])

I'm ready to assist you in improving a user query. Here's a hypothetical query for you to work with:

**Original Query:**
"I want something that can help me find a good restaurant near my location."

This query is brief but lacks specific details, making it challenging to provide a precise response. Your task is to improve and clarify this query.

Please suggest how you would rephrase or expand this query to make it more informative and easier to answer accurately.


LangFlow Equivalent Logic

## LangFlow Workflow Design

Flow Structure:

User Input
   ↓
Refinement Prompt
   ↓
ChatGroq Model
   ↓
Answer Generator Prompt
   ↓
Chat Output

This represents a two-step reasoning pipeline:
1. Query refinement
2. Final answer generation

LangFlow components required:
- Chat Input
- Prompt Template (Refiner)
- ChatGroq
- Prompt Template (Answer Generator)
- Chat Output

In [13]:
!pip install -U langflow


  Using cached langflow-1.7.3-py3-none-any.whl.metadata (12 kB)
  Using cached ddgs-9.10.0-py3-none-any.whl.metadata (12 kB)
  Using cached fastmcp-2.14.5-py3-none-any.whl.metadata (20 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langflow_base-0.7.3-py3-none-any.whl.metadata (4.5 kB)
  Using cached setuptools-80.10.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached uv-0.10.3-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached pypdf-6.7.0-py3-none-any.whl.metadata (7.1 kB)
INFO: pip is looking at multiple versions of ddgs to determine which version is compatible with other requirements. This could take a while.
  Using cached ddgs-9.9.3-py3-none-any.whl.metadata (19 kB)
  Using cached ddgs-9.9.2-py3-none-any.whl.metadata (19 kB)
  Using cached ddgs-9.9.1-py3-n

In [8]:
!python -m langflow run --host 0.0.0.0 --port 7860

/usr/local/lib/python3.12/dist-packages/langflow/main.py:20: FastAPIPaginationWarning: Pydantic v2.12.5 or higher is required for full compatibility. Please upgrade your Pydantic version to avoid potential issues.Support for Pydantic versions lower than v2.12.5 will be dropped in fastapi-pagination v0.16.0.
  from fastapi_pagination import add_pagination
✓ Initializing Langflow
✓ Checking Environment
■ Starting Core Services...2026-02-17T04:47:15.571754Z [warning  ] CORS: Using permissive defaults (all origins + credentials). Set LANGFLOW_CORS_ORIGINS for production. Stricter defaults in v2.0.
✓ Starting Core Services
✓ Connecting Database
✓ Loading Components
✓ Adding Starter Projects
▢ Launching Langflow...2026-02-17T04:47:16.586647Z [info     ] Initializing alembic          
□ Launching Langflow...2026-02-17T04:47:16.981601Z [warning  ] Column 'created_at' has type DATETIME in table 'apikey'
2026-02-17T04:47:16.982923Z [warning  ] Column 'created_at' has type DATETIME in table 'vari

In [14]:
from google.colab import output
output.serve_kernel_port_as_window(7860)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>